防災アプリAdventCalender2024 24日目の記事で作成した深発地震の予測モデル

### 事前準備

1. リポジトリから学習用パッケージをインストール
2. 簡易版学習用データ、予測地点データを用意

In [1]:
%pip install git+https://github.com/kotoho7/asid-predict

  Cloning https://github.com/kotoho7/asid-predict to /tmp/pip-req-build-gljyadpj
  Running command git clone --filter=blob:none --quiet https://github.com/kotoho7/asid-predict /tmp/pip-req-build-gljyadpj
  Resolved https://github.com/kotoho7/asid-predict to commit 66cf0a238e64f7600c8819bcde02593a877e47f6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 909.7/909.7 kB 18.5 MB/s eta 0:00:00
  Created wheel for asid_predict: filename=asid_predict-12.1.0-py3-none-any.whl size=24239 sha256=3c465d6dbe0bb769f15e21397a61ca98fa5f30f5349cab161583940490c187a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-rl1ayogq/wheels/d7/df/93/7bf53d2d54bbc919ad4687c236847fc24086ab43ff5eacb9ae
Successfully built asid_predict


In [2]:
!mkdir -p ./data ./out
!wget https://gist.githubusercontent.com/kotoho7/e1388671e5f990585e921f640b708872/raw/train_data_sample.json -O ./data/train_data.json
!wget https://gist.githubusercontent.com/kotoho7/e1388671e5f990585e921f640b708872/raw/kik.json -O ./data/predict.json

--2025-01-05 13:12:57--  https://gist.githubusercontent.com/kotoho7/e1388671e5f990585e921f640b708872/raw/train_data_sample.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 724081 (707K) [text/plain]
Saving to: ‘./data/train_data.json’

./data/train_data.j 100%[===================>] 707.11K  --.-KB/s    in 0.04s   

2025-01-05 13:12:57 (19.1 MB/s) - ‘./data/train_data.json’ saved [724081/724081]

--2025-01-05 13:12:57--  https://gist.githubusercontent.com/kotoho7/e1388671e5f990585e921f640b708872/raw/kik.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

### モデルの学習

In [ ]:
from asid_predict import execute_training_process

# モデルの学習
model = execute_training_process(
    train_json_path="./data/train_data.json", # 学習用データのパス
    save_path="./out", # モデルの保存先
    target_is_pasific_plate=True, # 太平洋プレートの地震を学習
    epochs=20, # エポック数
    batch_size=64, # バッチサイズ
)

### 学習済みモデルを使って予想震度を計算

In [ ]:
import json

from asid_predict.prediction import predict_intensities
from asid_predict.dataclass import Earthquake, ObservationPoint

# 予測地点リスト
with open("./data/predict.json") as f:
    dict = json.load(f)
    stations = [ObservationPoint(**d) for d in dict]

# 震源情報
earthquake = Earthquake(
    lat=33.840,
    lon=137.423,
    depth=362,
    magnitude=6.4,
)

# 予測を実行
intensities = predict_intensities(
    model=model,
    targets=stations,
    eq=earthquake,
)

### 予測結果の表示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x, y, c = np.array([s.lon for s in stations]), np.array([s.lat for s in stations]), np.array(intensities)
sorted_indices = np.argsort(c)

# 散布図を作成
plt.scatter(x[sorted_indices], y[sorted_indices], c=c[sorted_indices], cmap="jet", vmin=-3, vmax=7)
plt.colorbar(label="Seismic Intensity")
plt.scatter(earthquake.lon, earthquake.lat, color="red", marker="*", s=200, label="Epicenter")
plt.title(f"Earthquake at ({earthquake.lat}, {earthquake.lon}), Depth: {earthquake.depth} km, Magnitude: {earthquake.magnitude}")
plt.xlim(128, 146)
plt.ylim(30, 46)
plt.legend()
plt.show()